# Lab 2.4: LLM Observability with Arize Phoenix

In this lab, we will learn how to instrument our LLM applications for observability using **Arize Phoenix**. Observability is crucial for understanding how your LLM applications are performing, debugging issues, and evaluating quality.

We will cover:
1.  **Setup**: Installing Phoenix and launching the local UI.
2.  **Part 1: Simple LangChain Flow**: Instrumenting a basic chain.
3.  **Part 2: Advanced LangGraph Application**: Instrumenting a stateful agent workflow.

### Prerequisites
- Ensure you have your Groq API Key ready.


In [1]:
# 1. Install Dependencies
%pip install -qU arize-phoenix arize-otel openinference-instrumentation-langchain langchain langchain-groq langgraph
%pip install -q "arize[AutoEmbeddings]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 2. Setup API Keys
import getpass
import os
from arize.otel import register

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

if "ARIZE_SPACE_ID" not in os.environ:
    os.environ["ARIZE_SPACE_ID"] = getpass.getpass("Enter your Arize Space ID: ")

if "ARIZE_API_KEY" not in os.environ:
    os.environ["ARIZE_API_KEY"] = getpass.getpass("Enter your Arize API Key: ")


Enter your Groq API Key: ··········
Enter your Arize Space ID: ··········
Enter your Arize API Key: ··········


## 3. Launch Arize Phoenix (Optional)
You can still launch the Phoenix application locally to view traces locally, or you can view them in the Arize Cloud dashboard.

In [3]:
# Import Arize observability components
from arize.otel import register
from getpass import getpass

# Configure Arize tracing
tracer_provider = register(
    space_id=os.environ["ARIZE_SPACE_ID"],
    api_key=os.environ["ARIZE_API_KEY"],
    project_name="arize-lab-demo"
)

# Enable automatic LangChain instrumentation
from openinference.instrumentation.langchain import LangChainInstrumentor
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

print("🔭 Arize observability configured successfully!")
print("📊 All LangChain operations will now be automatically traced")

🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: arize-lab-demo
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

🔭 Arize observability configured successfully!
📊 All LangChain operations will now be automatically traced


## Part 1: Simple LangChain Flow (Banking Policy Assistant)

We will create a simple **Banking Policy Assistant** that answers internal policy questions. This simulates a basic RAG or Chatbot interaction.

In [4]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Setup a Banking Policy Assistant
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)

# This prompt simulates an internal knowledge base lookup
policy_prompt = ChatPromptTemplate.from_template(
    "You are a Banking Policy Assistant. Answer the question based on standard mortgage guidelines. Keep it professional. Question: {question}"
)
chain = policy_prompt | llm | StrOutputParser()

# Invoke the chain
print("Invoking Banking Policy Assistant...")
question = "What is the maximum loan-to-value (LTV) ratio for a jumbo mortgage?"
print(f"Question: {question}")
response = chain.invoke({"question": question})
print(f"Response: {response}")

# Check the Phoenix UI and Arize Cloud to see the trace!

Invoking Banking Policy Assistant...
Question: What is the maximum loan-to-value (LTV) ratio for a jumbo mortgage?
Response: The maximum loan-to-value (LTV) ratio for a jumbo mortgage typically ranges between **75% and 85%**, depending on the lender and the borrower's financial profile. Most lenders require a minimum down payment of **20% to 25%** for jumbo loans, resulting in an LTV of **80% to 85%**. However, borrowers with strong credit scores, stable income, and low debt-to-income ratios may qualify for higher LTVs in some cases. 

It is important to note that jumbo loans are not eligible for purchase by Fannie Mae or Freddie Mac and are subject to stricter underwriting standards due to their higher risk profile. Always confirm specific requirements with your lender, as policies can vary.


## Part 2: Advanced LangGraph Application (Loan Underwriting)

Now, let's look at a complex **Loan Underwriting Pipeline**. This involves multiple steps: Risk Analysis and Final Decision.

We will use LangGraph to model this stateful workflow, where a **Risk Analyst** assesses the applicant and a **Senior Underwriter** makes the final call.

In [5]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# --- 2. STATE DEFINITION ---
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    applicant_id: str
    risk_score: str

# Define Nodes
def risk_analyst_node(state: State):
    """Analyzes the financial health of the applicant."""
    applicant_data = state["messages"][-1].content

    system_msg = (
        "You are a Risk Analyst for a bank. Analyze the provided applicant financial data "
        "(Income, Debt, Credit Score). Calculate the Debt-to-Income (DTI) ratio. "
        "Summarize the financial health and assign a risk level (Low, Medium, High)."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("human", "Applicant Data: {data}")
    ])
    chain = prompt | llm
    response = chain.invoke({"data": applicant_data})

    # Extract a dummy risk score to update state (simplification)
    # In a real app, we'd use structured output
    return {"messages": [response], "risk_score": "Calculated"}

def senior_underwriter_node(state: State):
    """Makes the final loan decision based on risk analysis."""
    risk_analysis = state["messages"][-1].content

    system_msg = (
        "You are a Senior Underwriter. Review the Risk Analyst's report. "
        "Make a final decision: APPROVE or DENY. "
        "If approved, set the interest rate tier (Tier 1 is best). "
        "Provide a brief justification for the decision."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("human", "Risk Analysis Report: {report}")
    ])
    chain = prompt | llm
    response = chain.invoke({"report": risk_analysis})
    return {"messages": [response]}

# Build Graph
workflow = StateGraph(State)

# Add Nodes
workflow.add_node("risk_analyst", risk_analyst_node)
workflow.add_node("senior_underwriter", senior_underwriter_node)

# Define Logic
workflow.add_edge(START, "risk_analyst")
workflow.add_edge("risk_analyst", "senior_underwriter")
workflow.add_edge("senior_underwriter", END)

# Compile
graph = workflow.compile()

# Invoke Graph
print("🚀 Starting Loan Underwriting Pipeline...")

applicant_info = "Applicant: John Doe. Income: $150,000/yr. Monthly Debt: $2,500. Credit Score: 780. Loan Amount: $500,000."
print(f"Processing Application: {applicant_info}\n")

inputs = {
    "messages": [HumanMessage(content=applicant_info)],
    "applicant_id": "APP-12345"
}

for output in graph.stream(inputs, stream_mode="updates"):
    for node_name, node_state in output.items():
        print(f"\n--- NODE: {node_name.upper()} ---")
        last_msg = node_state["messages"][-1]
        print(f"Content: {last_msg.content[:300]}...")

print("\n✅ Pipeline Complete. Check Phoenix UI for the trace lineage.")

# Check Phoenix UI again. You should see a trace for this execution showing the graph steps.

🚀 Starting Loan Underwriting Pipeline...
Processing Application: Applicant: John Doe. Income: $150,000/yr. Monthly Debt: $2,500. Credit Score: 780. Loan Amount: $500,000.


--- NODE: RISK_ANALYST ---
Content: **Financial Analysis for John Doe**  

### **Key Calculations**  
1. **Debt-to-Income (DTI) Ratio**:  
   - **Monthly Income**: $150,000 / 12 = **$12,500**  
   - **DTI**: ($2,500 Monthly Debt / $12,500 Monthly Income) × 100 = **20%**  

2. **Loan-to-Income Ratio**:  
   - **Loan Amount / Annual Inc...

--- NODE: SENIOR_UNDERWRITER ---
Content: **Final Decision**: APPROVE  
**Interest Rate Tier**: Tier 1  

**Justification**:  
John Doe’s financial profile is exceptionally strong, with a low DTI (20%), excellent credit score (780), and high income ($150,000 annually). While the $500,000 loan is 3.33x his income, his robust repayment capaci...

✅ Pipeline Complete. Check Phoenix UI for the trace lineage.


## Conclusion
You have successfully set up Arize Phoenix and instrumented both a **Banking Policy Assistant** and a **Loan Underwriting Pipeline**.
Review the traces in the Phoenix UI to understand the internal execution of your Critical Financial Workflows.